In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings("ignore")

In [2]:
# uncomment if using colab
# ! git clone https://github.com/Daniel-Tran3/CSE_151A_Project.git

In [3]:
original_df = pd.read_csv('apartments_for_rent_classified_10K_utf.csv')

# uncomment the following if using colab
# original_df = pd.read_csv('CSE_151A_Project/apartments_for_rent_classified_10K_utf.csv')

In [4]:
df = original_df[["amenities", "bathrooms", "bedrooms", "fee", "price", "price_type", "square_feet", "cityname", "state", "time"]]

In [5]:
df.head(5)

,amenities,bathrooms,bedrooms,fee,price,price_type,square_feet,cityname,state,time
0,NaN,NaN,0.0,No,790,Monthly,101,Washington,DC,1577359415
1,NaN,NaN,1.0,No,425,Monthly,106,Evansville,IN,1577017063
2,NaN,1.0,0.0,No,1390,Monthly,107,Arlington,VA,1577359410
3,NaN,1.0,0.0,No,925,Monthly,116,Seattle,WA,1576667743
4,NaN,NaN,0.0,No,880,Monthly,125,Arlington,VA,1577359401


In [6]:
df.isna().mean()

amenities      0.3549
bathrooms      0.0034
bedrooms       0.0007
fee            0.0000
price          0.0000
price_type     0.0000
square_feet    0.0000
cityname       0.0077
state          0.0077
time           0.0000
dtype: float64

## Convert timestamp to datetime objects

In [7]:
df['time'] = pd.to_datetime(df['time'], unit='s')

In [8]:
df.head(2)

,amenities,bathrooms,bedrooms,fee,price,price_type,square_feet,cityname,state,time
0,NaN,NaN,0.0,No,790,Monthly,101,Washington,DC,2019-12-26 11:23:35
1,NaN,NaN,1.0,No,425,Monthly,106,Evansville,IN,2019-12-22 12:17:43


## Normalizing Price and Square Feet


Convert any prices measured in Weeks to Months by converting to price per day then price per year then price per month (roughly 4.348 multiplier).

There is one abnormally high price with 'weekly' price_type. The price for 'week' is 1560, and digging into the dataset, we found that it is a duplicate of index 15. The listing is for the same place except it's on another listing website. This duplicate will be dropped in favor of the other one.

There is also one record with 'Monthly|Weekely' price_type with a price of 275.0, and the price is likely to be on a weekly basis.

In [9]:
df[df['price'] == 1560][:2]

,amenities,bathrooms,bedrooms,fee,price,price_type,square_feet,cityname,state,time
15,"AC,Basketball,Cable or Satellite,Gym,Internet ...",1.0,1.0,No,1560,Weekly,200,New Bern,NC,2019-12-17 21:27:56
16,"AC,Basketball,Cable or Satellite,Gym,Internet ...",1.0,1.0,No,1560,Monthly,200,New Bern,NC,2019-12-15 10:37:53


In [10]:
# Drop the wrong record?
# df = df.drop([16,17])

df = df.drop([15,16])

In [11]:
df[df['price_type'] == 'Monthly|Weekly']

,amenities,bathrooms,bedrooms,fee,price,price_type,square_feet,cityname,state,time
235,"Cable or Satellite,Pool,Refrigerator,Storage,TV",NaN,NaN,No,275,Monthly|Weekly,300,Lakeland,FL,2019-11-27 21:43:20


In [52]:
# DO NOT NORMALIZE IT WILL BECOME NON-INTERPRETABLE
# for i in df.index:
#   if (df['price_type'][i] == "Weekly") or (df['price_type'][i] == "Monthly/Weekly"):
#     df.loc[i, 'price'] = df['price'][i] / 7 * 365.25 / 12

# df['price'] = (df['price'] - df['price'].min()) / (df['price'].max() - df['price'].min())

In [13]:
print(df['price'].min())
print(df['price'].max())

200
52500


Since we have standardized the units of the price, we can drop the price_type column.

In [14]:
df = df.drop(columns=['price_type'])

## Normalize the square_feet using Min Max method

In [15]:
df['square_feet'] = (df['square_feet'] - df['square_feet'].min()) / (df['square_feet'].max() - df['square_feet'].min())

In [16]:
print(df['square_feet'].min())
print(df['square_feet'].max())

0.0
1.0


## Impute NaN values using median

In [17]:
df['bathrooms'] = df['bathrooms'].replace({np.nan: df['bathrooms'].median()})

In [18]:
df['bedrooms'] = df['bedrooms'].replace({np.nan: df['bedrooms'].median()})

## Preprocessing on the amenities

In [19]:
df['amenities'] = np.array(df['amenities'].str.strip().str.split(","))

In [20]:
df

,amenities,bathrooms,bedrooms,fee,price,square_feet,cityname,state,time
0,NaN,1.0,0.0,No,790,0.000000,Washington,DC,2019-12-26 11:23:35
1,NaN,1.0,1.0,No,425,0.000125,Evansville,IN,2019-12-22 12:17:43
2,NaN,1.0,0.0,No,1390,0.000150,Arlington,VA,2019-12-26 11:23:30
3,NaN,1.0,0.0,No,925,0.000376,Seattle,WA,2019-12-18 11:15:43
4,NaN,1.0,0.0,No,880,0.000602,Arlington,VA,2019-12-26 11:23:21
...,...,...,...,...,...,...,...,...,...
9995,NaN,4.0,5.0,No,6000,0.155367,Edina,MN,2019-11-30 11:22:55
9996,NaN,8.0,6.0,No,25000,0.215920,Montecito,CA,2019-12-26 11:40:19
9997,NaN,8.5,6.0,No,11000,0.281135,Potomac,MD,2019-12-26 11:42:40
9998,"[Basketball, Cable or Satellite, Doorman, Hot ...",1.0,1.0,No,4790,1.000000,New York,NY,2019-12-26 12:09:46


In [21]:
df['amenities'] = df['amenities'].replace({np.nan: 'None'})

In [22]:
df.head(5)

,amenities,bathrooms,bedrooms,fee,price,square_feet,cityname,state,time
0,None,1.0,0.0,No,790,0.000000,Washington,DC,2019-12-26 11:23:35
1,None,1.0,1.0,No,425,0.000125,Evansville,IN,2019-12-22 12:17:43
2,None,1.0,0.0,No,1390,0.000150,Arlington,VA,2019-12-26 11:23:30
3,None,1.0,0.0,No,925,0.000376,Seattle,WA,2019-12-18 11:15:43
4,None,1.0,0.0,No,880,0.000602,Arlington,VA,2019-12-26 11:23:21


## One-hot Encoding States and City Names
States and city names are nominal categorical features. To deal with this, we will be creating one-hot encoded fields to reflect each data points location.

In [23]:
all_states = df['state'].unique()
all_states

array(['DC', 'IN', 'VA', 'WA', 'NY', 'CA', 'AZ', 'TX', 'GA', 'NC', 'FL',
       nan, 'AL', 'MD', 'CO', 'NM', 'IL', 'TN', 'AK', 'MA', 'NJ', 'OR',
       'DE', 'PA', 'IA', 'SC', 'MN', 'MI', 'KY', 'WI', 'OH', 'CT', 'RI',
       'NV', 'UT', 'MO', 'OK', 'NH', 'NE', 'LA', 'ND', 'AR', 'KS', 'ID',
       'HI', 'MT', 'VT', 'SD', 'WV', 'MS', 'ME', 'WY'], dtype=object)

In [24]:
display(df['state'].value_counts())

display(df['cityname'].value_counts())

TX    1737
CA     955
WA     519
NC     436
MD     424
NJ     383
GA     372
FL     339
OH     321
CO     318
WI     302
IL     282
MO     239
IN     239
MN     221
VA     205
OR     197
PA     183
IA     179
OK     178
MI     176
MA     167
AZ     126
NV     121
ND     113
NE     105
CT      98
TN      92
UT      84
KS      83
DC      80
SC      77
NY      71
NH      70
SD      66
LA      66
AL      56
AR      56
AK      44
KY      40
ID      21
VT      16
NM      14
HI      12
RI      11
MS       9
MT       7
DE       5
WV       3
ME       2
WY       1
Name: state, dtype: int64

Austin           523
Dallas           216
Houston          186
San Antonio      182
Los Angeles      165
                ... 
Keizer             1
Keyser             1
Pompano Beach      1
Kaysville          1
Bella Vista        1
Name: cityname, Length: 1573, dtype: int64

In [25]:
df = pd.get_dummies(df, columns=['state', 'cityname'], dummy_na=True)

In [26]:
df.head()

,amenities,bathrooms,bedrooms,fee,price,square_feet,time,state_AK,state_AL,state_AR,...,cityname_York,cityname_Yorktown,cityname_Yorkville,cityname_Youngstown,cityname_Youngsville,cityname_Ypsilanti,cityname_Yuba City,cityname_Yukon,cityname_Zachary,cityname_nan
0,None,1.0,0.0,No,790,0.000000,2019-12-26 11:23:35,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,None,1.0,1.0,No,425,0.000125,2019-12-22 12:17:43,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,None,1.0,0.0,No,1390,0.000150,2019-12-26 11:23:30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,None,1.0,0.0,No,925,0.000376,2019-12-18 11:15:43,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,None,1.0,0.0,No,880,0.000602,2019-12-26 11:23:21,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# extract unknown cities and states
unknown_city = df[df['cityname_nan'] == 1]
unknown_state = df[df['state_nan'] == 1]
unknown_state

# find if there is intersection between unknown cities and states
print(unknown_city.shape[0])
print(unknown_state.shape[0])
len(unknown_city.index.intersection(unknown_state.index))

77
77


77

In [28]:
df['fee'].unique()

array(['No'], dtype=object)

In [29]:
final_cleaned = df.drop(columns=['fee'])
final_cleaned

,amenities,bathrooms,bedrooms,price,square_feet,time,state_AK,state_AL,state_AR,state_AZ,...,cityname_York,cityname_Yorktown,cityname_Yorkville,cityname_Youngstown,cityname_Youngsville,cityname_Ypsilanti,cityname_Yuba City,cityname_Yukon,cityname_Zachary,cityname_nan
0,None,1.0,0.0,790,0.000000,2019-12-26 11:23:35,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,None,1.0,1.0,425,0.000125,2019-12-22 12:17:43,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,None,1.0,0.0,1390,0.000150,2019-12-26 11:23:30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,None,1.0,0.0,925,0.000376,2019-12-18 11:15:43,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,None,1.0,0.0,880,0.000602,2019-12-26 11:23:21,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,None,4.0,5.0,6000,0.155367,2019-11-30 11:22:55,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,None,8.0,6.0,25000,0.215920,2019-12-26 11:40:19,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,None,8.5,6.0,11000,0.281135,2019-12-26 11:42:40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,"[Basketball, Cable or Satellite, Doorman, Hot ...",1.0,1.0,4790,1.000000,2019-12-26 12:09:46,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## One hot encoding amenities

In [30]:
all_amenities = (final_cleaned['amenities'].explode()).unique()[1:]
final_cleaned = final_cleaned.reset_index(drop=True)
amenities = pd.DataFrame()

for a in all_amenities:
    amenities[a] = final_cleaned['amenities'].apply(lambda x: 1 if a in x else 0)
amenities.head()

,Dishwasher,Elevator,Patio/Deck,Pool,Storage,Refrigerator,Cable or Satellite,Clubhouse,Internet Access,Parking,...,Gym,TV,Hot Tub,Tennis,Wood Floors,View,Alarm,Doorman,Luxury,Golf
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
final_cleaned = pd.concat([final_cleaned, amenities], axis=1).drop(columns='amenities')
final_cleaned.head()

,bathrooms,bedrooms,price,square_feet,time,state_AK,state_AL,state_AR,state_AZ,state_CA,...,Gym,TV,Hot Tub,Tennis,Wood Floors,View,Alarm,Doorman,Luxury,Golf
0,1.0,0.0,790,0.000000,2019-12-26 11:23:35,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,1.0,425,0.000125,2019-12-22 12:17:43,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0.0,1390,0.000150,2019-12-26 11:23:30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,0.0,925,0.000376,2019-12-18 11:15:43,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,880,0.000602,2019-12-26 11:23:21,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Model 1

In [162]:
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

# drop time since machine cant train on DateTime type
X = final_cleaned.drop(columns=['price', 'time'])
X_train, X_test, y_train, y_test = train_test_split(X, final_cleaned.price, test_size = 0.2, random_state = 42)

def buildHPmodel(hp):
  model = Sequential ([
        Dense(
            units=hp.Int("units", min_value=8, max_value=64, step=8),
            input_shape=(X_train.shape[1],),
            activation=hp.Choice("activation", ["relu", "tanh"]),
        ),
      Dense(
          units = hp.Int("units", min_value = 8, max_value = 64, step = 8),
          activation = hp.Choice("activation", ["relu", "tanh"]),
      ),
      Dense(1, activation="relu")
  ])
  learning_rate = hp.Float("lr", min_value = 1e-4, max_value = 0.3, sampling = "log")
  model.compile(optimizer = SGD(learning_rate = learning_rate), loss = 'mae', metrics = ['mse','mae'])
  return model

In [155]:
y_train

8964    1862
1561     850
1670    1220
4345     800
8099    1300
        ... 
5734    1010
5191    1736
5390    3159
860     1061
7270    1285
Name: price, Length: 7998, dtype: int64

In [156]:
# pip install keras_tuner

In [157]:
import keras_tuner

In [158]:
hp = keras_tuner.HyperParameters()

In [159]:
# if changing any model parameters such as layers, activation function.
# plz remove the file "untitled_project/tuner0.json"
# otherwise, it will reload from the tuner config before modification.
tuner = keras_tuner.RandomSearch(
    hypermodel = buildHPmodel,
    objective = "val_mae",
    max_trials = 20,
    seed = 15,
    executions_per_trial = 5,
    tune_new_entries = True,
    allow_new_entries = True,
    max_consecutive_failed_trials = 3
)

In [160]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state = 0)

In [161]:
tuner.search(X_train.astype('float'), y_train, epochs = 2, validation_data = (X_val, y_val))

Trial 20 Complete [00h 00m 09s]
val_mae: 422.9543823242187

Best val_mae So Far: 411.8282775878906
Total elapsed time: 00h 02m 57s


In [163]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_mae", direction="min")

Trial 15 summary
Hyperparameters:
units: 24
activation: relu
lr: 0.007177664122743156
Score: 411.8282775878906

Trial 19 summary
Hyperparameters:
units: 8
activation: relu
lr: 0.005241515692754444
Score: 422.9543823242187

Trial 03 summary
Hyperparameters:
units: 48
activation: relu
lr: 0.0031230758528628527
Score: 492.50872802734375

Trial 13 summary
Hyperparameters:
units: 64
activation: relu
lr: 0.008547485565344062
Score: 654.8868774414062

Trial 17 summary
Hyperparameters:
units: 8
activation: relu
lr: 0.00304325168554751
Score: 787.1304626464844

Trial 02 summary
Hyperparameters:
units: 16
activation: tanh
lr: 0.007075066125528888
Score: 1468.967333984375

Trial 01 summary
Hyperparameters:
units: 16
activation: tanh
lr: 0.006475590128522282
Score: 1474.0741455078125

Trial 07 summary
Hyperparameters:
units: 24
activation: tanh
lr: 0.0019008479966982695
Score: 1498.5926

In [165]:
# test regular
model = Sequential()

# Input layer (matches the number of features in your data)
model.add(Dense(24, input_dim=X_train.shape[1], activation='relu')) 

# Hidden layers (you can add more with different units)
model.add(Dense(24, activation='relu'))  

# Output layer (units based on the number of classes)
model.add(Dense(1, activation='relu')) 

model.compile(optimizer='adam', loss='mae', metrics=['mae','mse'])

X_train, X_test, y_train, y_test = train_test_split(X, final_cleaned.price, test_size = 0.2)

model.fit(X_train,y_train, epochs=100, validation_data=(X_test, y_test))
loss, mae, mse = model.evaluate(X_test, y_test)
print('MAE:', mae)
print('MSE:', mse)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 1450.3462 - mae: 1450.3462 - mse: 3363610.0000 - val_loss: 1325.6533 - val_mae: 1325.6533 - val_mse: 2500059.7500
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 983.8949 - mae: 983.8949 - mse: 2234572.2500 - val_loss: 670.5816 - val_mae: 670.5816 - val_mse: 1039568.8125
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 602.3536 - mae: 602.3536 - mse: 1361513.6250 - val_loss: 531.4960 - val_mae: 531.4960 - val_mse: 723178.3750
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 503.9961 - mae: 503.9961 - mse: 1149204.7500 - val_loss: 452.2851 - val_mae: 452.2851 - val_mse: 575816.3750
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 443.7537 - mae: 443.7537 - mse: 1026729.3750 - val_loss: 406.5997 - val_mae: 406.5997 - val_mse: 491576.8125
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss:

250/250 [==============================] - 0s 2ms/step - loss: 257.6401 - mae: 257.6401 - mse: 677646.6875 - val_loss: 283.4389 - val_mae: 283.4389 - val_mse: 246160.7500
Epoch 46/100
250/250 [==============================] - 0s 2ms/step - loss: 256.8255 - mae: 256.8255 - mse: 676444.2500 - val_loss: 282.9951 - val_mae: 282.9951 - val_mse: 246385.1094
Epoch 47/100
250/250 [==============================] - 0s 2ms/step - loss: 256.0901 - mae: 256.0901 - mse: 674665.1875 - val_loss: 282.9431 - val_mae: 282.9431 - val_mse: 245025.8750
Epoch 48/100
250/250 [==============================] - 0s 2ms/step - loss: 255.2106 - mae: 255.2106 - mse: 673120.3750 - val_loss: 282.6294 - val_mae: 282.6294 - val_mse: 242803.8438
Epoch 49/100
250/250 [==============================] - 0s 2ms/step - loss: 254.4689 - mae: 254.4689 - mse: 671504.0625 - val_loss: 282.0252 - val_mae: 282.0252 - val_mse: 244651.9375
Epoch 50/100
250/250 [==============================] - 0s 2ms/step - loss: 253.7875 - mae: 2

250/250 [==============================] - 0s 2ms/step - loss: 235.8278 - mae: 235.8278 - mse: 628146.7500 - val_loss: 276.2091 - val_mae: 276.2091 - val_mse: 223023.1094
Epoch 90/100
250/250 [==============================] - 0s 2ms/step - loss: 235.4388 - mae: 235.4388 - mse: 627265.9375 - val_loss: 276.1469 - val_mae: 276.1469 - val_mse: 223191.3750
Epoch 91/100
250/250 [==============================] - 0s 2ms/step - loss: 235.0930 - mae: 235.0930 - mse: 626979.6875 - val_loss: 275.7753 - val_mae: 275.7753 - val_mse: 222126.7500
Epoch 92/100
250/250 [==============================] - 0s 1ms/step - loss: 234.6437 - mae: 234.6437 - mse: 626170.8750 - val_loss: 275.8242 - val_mae: 275.8242 - val_mse: 222633.9062
Epoch 93/100
250/250 [==============================] - 0s 1ms/step - loss: 234.4279 - mae: 234.4279 - mse: 625378.9375 - val_loss: 275.5784 - val_mae: 275.5784 - val_mse: 221860.0469
Epoch 94/100
250/250 [==============================] - 0s 2ms/step - loss: 234.0147 - mae: 2

The testing MAE is slightly worse than the training MAE but the difference isn't big enough to say that it is overfitting or underfitting. 

Conclusions:

ReLu is by far the best activation function for our task, which make sense since there are linear relationships in the data.

# Next Models: Linear Regression and Gradient Boosting Machines